In [1]:
import numpy as np
import pandas as pd

from sklearn import tree
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics
from sklearn import ensemble
from sklearn.decomposition import PCA

/home/pavle/anaconda3/envs/ml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/pavle/anaconda3/envs/ml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/pavle/anaconda3/envs/ml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
train_data = np.array(pd.read_csv('./data/train_data.csv'))
X_train_val = train_data[:, :-1]
y_train_val = train_data[:, -1]

test_data = np.array(pd.read_csv('./data/test_data.csv'))
X_test = test_data[:, :-1]
y_test = test_data[:, -1]

number_of_games_in_16_17 = 1074

x_train = X_train_val[:-number_of_games_in_16_17, :]
x_validation = X_train_val[-number_of_games_in_16_17:, :]

y_train = y_train_val[:-number_of_games_in_16_17]
y_validation = y_train_val[-number_of_games_in_16_17:]

In [3]:
scaler_tv = preprocessing.StandardScaler()
scaler_tv.fit(x_train)
x_train = scaler_tv.transform(x_train)
x_validation = scaler_tv.transform(x_validation)

scaler_tt = preprocessing.StandardScaler()
scaler_tt.fit(X_train_val)
X_train_val = scaler_tt.transform(X_train_val)
X_test = scaler_tt.transform(X_test)

In [4]:
estimators = [10, 15, 20, 25, 30, 40, 50]
depths = [2, 3, 5]
variance = np.arange(5, 10) / 10
best_score = 0
best_params = {'n_estimators': 0, 'max_depth': 0, 'var':0}
for var in variance:
    x_tr = x_train
    x_val = x_validation
    pca = PCA(var)
    pca.fit(x_tr)
    x_tr = pca.transform(x_tr)
    x_val = pca.transform(x_val)
    for n_estimators in estimators:
        for depth in depths:
            clf = ensemble.AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=depth),
                                              algorithm="SAMME", n_estimators=n_estimators, random_state=7)
            clf.fit(x_tr, y_train)
            y_predicted = clf.predict(x_val)
            score = metrics.accuracy_score(y_validation, y_predicted)
            if score > best_score: 
                best_score = score
                best_params['n_estimators'] = n_estimators
                best_params['max_depth'] = depth
                best_params['var'] = var

In [5]:
best_params

{'n_estimators': 15, 'max_depth': 3, 'var': 0.9}

In [6]:
pca = PCA(best_params['var'])
pca.fit(X_train_val)
X_train_val = pca.transform(X_train_val)
X_test = pca.transform(X_test)

In [7]:
clf = ensemble.AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=best_params['max_depth']),
                                 algorithm="SAMME", n_estimators=best_params['n_estimators'], random_state=7)

In [8]:
clf.fit(X_train_val, y_train_val)
y_predicted = clf.predict(X_test)
metrics.accuracy_score(y_test, y_predicted)

0.6465116279069767